# 3. BERT

Inicialmente se va a realizar un preprocesado de los datos, eliminando ls palabras sin significado útil, los url y los signos de puntuación.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/train.csv')

In [2]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [3]:
en_stopwords = nlp.Defaults.stop_words

def remove_stop_words(text):
    cleanText = ''
    phrase = nlp(text)
    for token in phrase:
        if not token.is_stop and not token.is_punct and not token.like_url:
            cleanText += ' ' + token.text

    return cleanText

df['text_cleaned'] = df['text'].apply(remove_stop_words)

In [ ]:
df['text_cleaned']

In [10]:
from sklearn.model_selection import train_test_split

X = df['text_cleaned']
y = df['label']

X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

X_tokizer = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors="tf")
Y = tf.convert_to_tensor(y_train)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss)

model.fit(X_tokizer, Y, epochs=3, batch_size=8)

predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, predictions))

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_test, predictions)